# Scratch book to try out different functions

## 1. Read lines in move_lines to build a dict of lines

In [1]:
from data import TextReader
from word2vec import word2vec

Using CUDA


In [2]:
file_name = "./Data/cornell_data/movie_lines.txt"
reader = TextReader()
reader.read_line_dict(file_name)

In [3]:
# output all movie lines without indices
reader.output_pure_lines()

In [ ]:
# The first line stored in movie_lines.txt
print(reader.lines[1045])
# Find the longest line (used in seq2seq for attention)
max_length = 0
max_line   = None
max_key    = 0
for key in reader.lines.keys():
    line = reader.lines[key]
    length = len(line.split(" "))
    if length > max_length:
        max_length = length
        max_line   = line
        max_key    = key

print((max_length, max_key, max_line))

## 2. Get vocabulary

In [ ]:
from collections import defaultdict
vocab = defaultdict(int)
for index in reader.lines.keys():
    line = reader.lines[index]
    words = line.split()
    for word in words:
        vocab[word] += 1

print(len(vocab))
print(vocab['they'])

## 3. Read dialogues

In [ ]:
# Default file path is movie_conversations
reader.read_dialogues()
print(reader.dialogues[0])

## 4. Sanity Test on Word2Vec Prep

In [4]:
from data import TextReader
from word2vec import word2vec

In [5]:
# Getting word_model embedding
word_model = word2vec()
word_model.fit("/home/aufish/Documents/19SP/NeuralBot_2/Data/cornell_data/pure_movie_lines.txt")
word_model.save()

/home/aufish/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/aufish/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## 5. Sanity Test for EncoderRNN

In [ ]:
from seq2seq import EncoderRNN, AttnDecoderRNN
import torch

In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:1")      # index may be different

In [ ]:
import torch

HIDDEN_SIZE = 5
INPUT_SIZE  = 12
BATCH_SIZE  = 10

encoder = EncoderRNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE)
decoder = AttnDecoderRNN(hidden_size=HIDDEN_SIZE, output_size=INPUT_SIZE, batch_size=BATCH_SIZE)

# sequence length 5
input_tensor = torch.randn(5, BATCH_SIZE, INPUT_SIZE).to(device)
(output, hidd) = encoder(input_tensor, encoder.initHidden(batch_size=BATCH_SIZE))

In [ ]:
output.shape

In [ ]:
decoding_start = torch.randn(1, BATCH_SIZE, INPUT_SIZE).to(device)
decoded = decoder.forward(decoding_start, decoder.initHidden(), output)

In [ ]:
print(decoded)

## 6. Brief Test on Training

In [11]:
import torch
import random
from word2vec import *
# Teaching forcing is to use the target as next input instead of 
# model output to correct the training online
teacher_forcing_ratio = 0.5

device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")

max_length = 100
# Training function on one batch
def train(x_tensor, t_tensor, encoder, decoder, en_optimizer, 
         de_optimizer, criterion, word_model):
    encoder_hidden = encoder.initHidden()
#     encoder_hidden = (encoder_hidden[0].to(device), encoder_hidden[1].to(device))
    
    en_optimizer.zero_grad()
    de_optimizer.zero_grad()
    
    input_length  = x_tensor.size(0) # The first dimension is seq length
    target_length = t_tensor.size(0)
    batch_size    = x_tensor.size(1)
    dimension     = x_tensor.size(2)
    
    encoder_outputs = \
        torch.zeros((max_length, batch_size, encoder.hidden_size), device=device)
    
    loss = 0
    
    for index in range(input_length):
        (encoder_y, encoder_hidden) = encoder(x_tensor[index:index+1], encoder_hidden)
        encoder_outputs[index]      = encoder_y[0]  # Pending confirmation
        
    decoder_input = torch.zeros((1, batch_size, dimension), device=device)
    for i in range(batch_size):
        decoder_input[0, i] = word_model.transform([START])
    decoder_hidden = decoder.initHidden()
#     decoder_hidden = (decoder_hidden[0].to(device), decoder_hidden[1].to(device))
    
    use_teacher_forcing = True \
        if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Feed the target as the next input
        for index in range(target_length):
            (decoder_y, decoder_hidden, attn_weights) = \
            decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_y[0], t_tensor[index])
            decoder_input = t_tensor[index]
    else:
        for index in range(target_length):
            (decoder_y, decoder_hidden, attn_weights) = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_y[0], t_tensor[index])
            
    loss.backward()
    
    en_optimizer.step()
    de_optimizer.step()
    
    return loss.item() / target_length

In [12]:
from data import TextReader
from seq2seq import *
from torch.optim import Adam
from torch.nn import MSELoss

model  = word2vec()
model.load()

/home/aufish/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
sentence1 = "Hi what's your name?".split()
sentence2 = "Tracer".split()

(input_tensor, target_tensor) = model.transform_pair(sentence1, sentence2)
print((input_tensor.size(), target_tensor.size()))

(torch.Size([4, 500]), torch.Size([1, 500]))


In [14]:
input_tensor = (input_tensor.view(input_tensor.size(0), 1, input_tensor.size(1)))

encoder = EncoderRNN(batch_size=1, device=device)
decoder = AttnDecoderRNN(batch_size=1, device=device)

en_optimizer = Adam(encoder.parameters(), lr=0.001)
de_optimizer = Adam(decoder.parameters(), lr=0.001)
criterion    = MSELoss()

train(input_tensor, target_tensor, encoder, decoder, 
      en_optimizer, de_optimizer, criterion, model)

(torch.Size([100, 1, 300]), torch.Size([500, 1, 300]))


38.64091110229492

## 7. Check if embedded model may be propagated

In [15]:
from seq2seq import *
class test_model(nn.Module):
    def __init__(self):
        super(test_model, self).__init__()
        self.encoder = EncoderRNN(batch_size=1)
        
    def forward(self, input_tensor):
        output, _ = self.encoder(input_tensor, self.encoder.initHidden())
        return output

In [23]:
sentence1 = "Hi what's your name?".split()

input_tensor = model.transform(sentence1)
input_tensor = (input_tensor.view(input_tensor.size(0), 1, input_tensor.size(1)))

t_model = test_model()

In [25]:
target_tensor = torch.zeros((4, 1, 300), device=device)

print(output.size())
print(target_tensor.size())

torch.Size([4, 1, 300])
torch.Size([4, 1, 300])


In [38]:
optimizer = Adam(t_model.parameters())
criterion = MSELoss()

output   = t_model(input_tensor)
print(output)
loss = criterion(output, target_tensor)
print(type(loss))
loss.backward()
optimizer.step()
output_2 = t_model(input_tensor)
print(output_2)

tensor([[[ 0.0169, -0.0130,  0.0868,  ..., -0.0112,  0.0217,  0.0231]],

        [[-0.0037, -0.0337,  0.0520,  ...,  0.0024, -0.0164,  0.1498]],

        [[ 0.0163, -0.0585,  0.0325,  ..., -0.0369, -0.0380, -0.0079]],

        [[-0.0097, -0.0557, -0.0086,  ..., -0.1367, -0.0739,  0.0959]]],
       device='cuda:1', grad_fn=<CudnnRnnBackward>)
<class 'torch.Tensor'>
tensor([[[ 0.0002,  0.0064,  0.0905,  ..., -0.0303,  0.0257,  0.0275]],

        [[-0.0256, -0.0085,  0.0608,  ..., -0.0172,  0.0009,  0.1627]],

        [[ 0.0144, -0.0393,  0.0292,  ..., -0.0401, -0.0300,  0.0117]],

        [[-0.0287, -0.0391, -0.0274,  ..., -0.1509, -0.0652,  0.1007]]],
       device='cuda:1', grad_fn=<CudnnRnnBackward>)


In [51]:
# Check parameters before
a = torch.tensor((1.0,2))
b = torch.tensor((1.0,2))
torch.log(a.dot(b))

tensor(1.6094)